In [1]:
!pip install netCDF4
!pip install numpy==1.21.6
import numpy as np
import pandas as pd
import os
import netCDF4
import pathlib
from netCDF4 import Dataset
import xarray as xr


#File cleanup and conversion
#Specific file directories will have to be changed!

files = []
for file in os.listdir('/home/jovyan/Final Project/complete_data_raw'):
    if(file.endswith('.nc4')):
        files = files + ['/home/jovyan/Final Project/complete_data_raw/' + file]

    
    
xr = xr.open_mfdataset(files)
xr.to_netcdf('complete_data_2.nc', 'w')
data = Dataset('complete_data_2.nc')

  Using cached netCDF4-1.6.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.1 MB)
  Using cached cftime-1.6.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached numpy-1.21.6-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3


In [2]:
x = data.variables.keys()
print(x)

dict_keys(['time', 'time_bnds', 'X', 'Y', 'Psurf_f_tavg', 'Qair_f_tavg', 'Tair_f_tavg', 'Wind_f_tavg'])


In [3]:
#Creating Area Groups for Lat and Lon
lon = data.variables['Y'][:]
print(lon)
lat = data.variables['X'][:]
print(lat)

groups = []

for Y in lon:
    for X in lat:
        groups += [(round(Y,2), round(X,2))]
        
print(groups)
print(len(groups))

group_names_temp = []
count = 1
for pair in groups:
    group_names_temp += ["Area " + str(count) + " Temp(K)"]
    count += 1

[41.05 41.15 41.25 41.35 41.45 41.55 41.65 41.75 41.85 41.95 42.05]
[-73.65 -73.55 -73.45 -73.35 -73.25 -73.15 -73.05 -72.95 -72.85 -72.75
 -72.65 -72.55 -72.45 -72.35 -72.25 -72.15 -72.05 -71.95 -71.85]
[(41.05, -73.65), (41.05, -73.55), (41.05, -73.45), (41.05, -73.35), (41.05, -73.25), (41.05, -73.15), (41.05, -73.05), (41.05, -72.95), (41.05, -72.85), (41.05, -72.75), (41.05, -72.65), (41.05, -72.55), (41.05, -72.45), (41.05, -72.35), (41.05, -72.25), (41.05, -72.15), (41.05, -72.05), (41.05, -71.95), (41.05, -71.85), (41.15, -73.65), (41.15, -73.55), (41.15, -73.45), (41.15, -73.35), (41.15, -73.25), (41.15, -73.15), (41.15, -73.05), (41.15, -72.95), (41.15, -72.85), (41.15, -72.75), (41.15, -72.65), (41.15, -72.55), (41.15, -72.45), (41.15, -72.35), (41.15, -72.25), (41.15, -72.15), (41.15, -72.05), (41.15, -71.95), (41.15, -71.85), (41.25, -73.65), (41.25, -73.55), (41.25, -73.45), (41.25, -73.35), (41.25, -73.25), (41.25, -73.15), (41.25, -73.05), (41.25, -72.95), (41.25, -72.8

In [4]:
#Creating Empty Data Frame
start_date = '2010-01-01'
end_date = '2022-01-01'
date_range = pd.date_range(start = start_date, end = end_date, freq='MS')
df_temp = pd.DataFrame(0.0,columns = group_names_temp, index = date_range)
print(date_range)

DatetimeIndex(['2010-01-01', '2010-02-01', '2010-03-01', '2010-04-01',
               '2010-05-01', '2010-06-01', '2010-07-01', '2010-08-01',
               '2010-09-01', '2010-10-01',
               ...
               '2021-04-01', '2021-05-01', '2021-06-01', '2021-07-01',
               '2021-08-01', '2021-09-01', '2021-10-01', '2021-11-01',
               '2021-12-01', '2022-01-01'],
              dtype='datetime64[ns]', length=145, freq='MS')


In [5]:
#Filling in values from netCDF file into dataframe for surface air temperature

temp = data.variables['Tair_f_tavg']

rows = np.arange(0, data.variables['time'].size)
y_range = np.arange(0, lon.size)
x_range = np.arange(0, lat.size)

for row_num in rows:
    count = 0
    while count != 209:
        for Y in y_range:
            for X in x_range:
                df_temp.iloc[row_num][count] = temp[row_num][Y][X]
                count += 1

print(df_temp)        

/opt/conda/lib/python3.9/site-packages/pandas/core/series.py:977: UserWarning: Warning: converting a masked element to nan.
  values[key] = value


            Area 1 Temp(K)  Area 2 Temp(K)  Area 3 Temp(K)  Area 4 Temp(K)  \
2010-01-01      271.286102      271.657257             NaN             NaN   
2010-02-01      271.263367      271.612183             NaN             NaN   
2010-03-01      278.871948      279.100311             NaN             NaN   
2010-04-01      284.046814      284.213501             NaN             NaN   
2010-05-01      288.943268      289.072845             NaN             NaN   
...                    ...             ...             ...             ...   
2021-09-01      293.403717      293.736664             NaN             NaN   
2021-10-01      288.864563      289.198822             NaN             NaN   
2021-11-01      280.197113      280.640778             NaN             NaN   
2021-12-01      277.263123      277.663666             NaN             NaN   
2022-01-01      270.749115      271.191528             NaN             NaN   

            Area 5 Temp(K)  Area 6 Temp(K)  Area 7 Temp(K)  Are

In [6]:
print(rows)

#Convert temperature dataframe to csv
df_temp.to_csv("surface_air_temperature.csv", na_rep=np.NaN)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144]


In [7]:
#Creating data frame for surface presssure with new group name

group_names_pressure = []
count = 1
for pair in groups:
    group_names_pressure += ["Area " + str(count) + " Pressure(Pa)"]
    count += 1

df_pressure = pd.DataFrame(0.0, columns = group_names_pressure, index = date_range)


#Filling in values from netCDF file into dataframe for surface pressure

pressure = data.variables['Psurf_f_tavg']
print(pressure)

rows = np.arange(0, data.variables['time'].size)
y_range = np.arange(0, lon.size)
x_range = np.arange(0, lat.size)

for row_num in rows:
    count = 0
    while count != 209:
        for Y in y_range:
            for X in x_range:
                df_pressure.iloc[row_num][count] = pressure[row_num][Y][X]
                count += 1

print(df_pressure)   

<class 'netCDF4._netCDF4.Variable'>
float32 Psurf_f_tavg(time, Y, X)
    long_name: surface pressure
    _FillValue: -9999.0
    standard_name: surface_air_pressure
    vmax: 110000.0
    cell_methods: time:mean
    missing_value: -9999.0
    units: Pa
    vmin: 5000.0
unlimited dimensions: 
current shape = (145, 11, 19)
filling on


/opt/conda/lib/python3.9/site-packages/pandas/core/series.py:977: UserWarning: Warning: converting a masked element to nan.
  values[key] = value


            Area 1 Pressure(Pa)  Area 2 Pressure(Pa)  Area 3 Pressure(Pa)  \
2010-01-01        100655.046875        101183.468750                  NaN   
2010-02-01        100064.554688        100588.625000                  NaN   
2010-03-01        100381.898438        100899.531250                  NaN   
2010-04-01        100532.132812        101041.890625                  NaN   
2010-05-01        100781.695312        101285.687500                  NaN   
...                         ...                  ...                  ...   
2021-09-01        100737.929688        101233.398438                  NaN   
2021-10-01        100738.781250        101241.617188                  NaN   
2021-11-01        101012.257812        101528.570312                  NaN   
2021-12-01        100917.148438        101439.648438                  NaN   
2022-01-01        100961.890625        101494.554688                  NaN   

            Area 4 Pressure(Pa)  Area 5 Pressure(Pa)  Area 6 Pressure(Pa)  

In [8]:
#Convert pressure dataframe to csv
df_pressure.to_csv("surface_pressure.csv", na_rep=np.NaN)

In [9]:
#Creating data frame for wind speed with new group name

group_names_wind = []
count = 1
for pair in groups:
    group_names_wind += ["Area " + str(count) + " Wind Speed(m s-1)"]
    count += 1

df_wind = pd.DataFrame(0.0, columns = group_names_wind, index = date_range)


#Filling in values from netCDF file into dataframe for wind

wind = data.variables['Wind_f_tavg']
print(wind)

rows = np.arange(0, data.variables['time'].size)
y_range = np.arange(0, lon.size)
x_range = np.arange(0, lat.size)

for row_num in rows:
    count = 0
    while count != 209:
        for Y in y_range:
            for X in x_range:
                df_wind.iloc[row_num][count] = wind[row_num][Y][X]
                count += 1

print(df_wind)   

<class 'netCDF4._netCDF4.Variable'>
float32 Wind_f_tavg(time, Y, X)
    long_name: wind speed
    _FillValue: -9999.0
    standard_name: wind_speed
    vmax: 1000000000000000.0
    cell_methods: time:mean
    missing_value: -9999.0
    units: m s-1
    vmin: -1000000000000000.0
unlimited dimensions: 
current shape = (145, 11, 19)
filling on


/opt/conda/lib/python3.9/site-packages/pandas/core/series.py:977: UserWarning: Warning: converting a masked element to nan.
  values[key] = value


            Area 1 Wind Speed(m s-1)  Area 2 Wind Speed(m s-1)  \
2010-01-01                  6.435525                  6.531234   
2010-02-01                  6.854457                  6.935881   
2010-03-01                  6.694638                  6.744306   
2010-04-01                  5.520395                  5.542546   
2010-05-01                  5.205482                  5.263883   
...                              ...                       ...   
2021-09-01                  5.113967                  5.176341   
2021-10-01                  5.152925                  5.254683   
2021-11-01                  5.298468                  5.406521   
2021-12-01                  5.425148                  5.533946   
2022-01-01                  6.552473                  6.674996   

            Area 3 Wind Speed(m s-1)  Area 4 Wind Speed(m s-1)  \
2010-01-01                       NaN                       NaN   
2010-02-01                       NaN                       NaN   
2010-03-0

In [10]:
#Convert wind dataframe to csv
df_wind.to_csv("near_surface_wind_speed.csv", na_rep=np.NaN)

In [11]:
#Creating data frame for specific humidity with new group name

group_names_humidity = []
count = 1
for pair in groups:
    group_names_humidity += ["Area " + str(count) + " Specific Humidity (kg kg-1)"]
    count += 1

df_humidity = pd.DataFrame(0.0, columns = group_names_humidity, index = date_range)

#Filling in values from netCDF file into dataframe for specific humidity

humidity = data.variables['Qair_f_tavg']
print(humidity)

rows = np.arange(0, data.variables['time'].size)
y_range = np.arange(0, lon.size)
x_range = np.arange(0, lat.size)

for row_num in rows:
    count = 0
    while count != 209:
        for Y in y_range:
            for X in x_range:
                df_humidity.iloc[row_num][count] = humidity[row_num][Y][X]
                count += 1

print(df_humidity)

<class 'netCDF4._netCDF4.Variable'>
float32 Qair_f_tavg(time, Y, X)
    long_name: specific humidity
    _FillValue: -9999.0
    standard_name: specific_humidity
    vmax: 0.03
    cell_methods: time:mean
    missing_value: -9999.0
    units: kg kg-1
    vmin: 0.0
unlimited dimensions: 
current shape = (145, 11, 19)
filling on


/opt/conda/lib/python3.9/site-packages/pandas/core/series.py:977: UserWarning: Warning: converting a masked element to nan.
  values[key] = value


            Area 1 Specific Humidity (kg kg-1)  \
2010-01-01                            0.002711   
2010-02-01                            0.002661   
2010-03-01                            0.004474   
2010-04-01                            0.005903   
2010-05-01                            0.008316   
...                                        ...   
2021-09-01                            0.011416   
2021-10-01                            0.008861   
2021-11-01                            0.004746   
2021-12-01                            0.004015   
2022-01-01                            0.002652   

            Area 2 Specific Humidity (kg kg-1)  \
2010-01-01                            0.002763   
2010-02-01                            0.002704   
2010-03-01                            0.004532   
2010-04-01                            0.005964   
2010-05-01                            0.008364   
...                                        ...   
2021-09-01                            0.011568   

In [12]:
#Convert humidity dataframe to csv
df_humidity.to_csv("specific_humidity.csv", na_rep=np.NaN)